In [7]:
import numpy as np

def validate_input(A, b):
    # Перевірка типів даних
    if not (isinstance(A, np.ndarray) and isinstance(b, np.ndarray)):
        raise ValueError("Вхідні дані мають бути у вигляді numpy масивів")
    if not (np.issubdtype(A.dtype, np.number)
        and np.issubdtype(b.dtype, np.number)):
        raise ValueError("Матриця A і вектор b мають \
        містити лише числові значення")

    # Перевірка розмірів матриці A та вектора b
    if A.shape[0] != A.shape[1]:
        raise ValueError("Матриця A повинна бути квадратною")
    if A.shape[0] != b.shape[0]:
        raise ValueError("Кількість рядків у A повинна \
        дорівнювати кількості елементів у b")

    # Перевірка на виродженість матриці
    if np.linalg.det(A) == 0:
        raise ValueError("Матриця A є виродженою, \
        і система може не мати унікального розв'язку")

def gauss_method(A, b):
    validate_input(A, b)
    n = len(b)

    # Копіюємо A та b для уникнення зміни вхідних даних
    A = A.astype(float).copy()
    b = b.astype(float).copy()

    # Прямий хід
    for k in range(n):
        if abs(A[k, k]) < 1e-10:
            raise ValueError("Нульовий елемент на діагоналі; \
            система можливо вироджена або вимагає перестановки рядків")

        for i in range(k + 1, n):
            factor = A[i, k] / A[k, k]
            A[i, k:] -= factor * A[k, k:]
            b[i] -= factor * b[k]

    # Зворотний хід
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

    return x

def lu_factorization(A, b):
    validate_input(A, b)
    n = A.shape[0]

    # Ініціалізація матриць L та U
    L = np.eye(n)
    U = A.astype(float).copy()

    # LU факторизація
    for k in range(n):
        if abs(U[k, k]) < 1e-10:
            raise ValueError("Нульовий елемент на діагоналі; \
            система можливо вироджена або вимагає перестановки рядків")

        for i in range(k + 1, n):
            factor = U[i, k] / U[k, k]
            L[i, k] = factor
            U[i, k:] -= factor * U[k, k:]

    # Розв’язування L * y = b
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])

    # Розв’язування U * x = y
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]

    return x

def calculate_error(A, x, b):
    # Обчислення похибки як норма вектора нев’язки
    residual = np.dot(A, x) - b
    error = np.linalg.norm(residual)
    return error

def calculate_coordinate_accuracy(exact_solution, approx_solution):
    # Обчислення покоординатної точності
    coordinate_errors = np.abs(exact_solution - approx_solution)
    return coordinate_errors

# Вхідні дані
A = np.array([
    [1.02, -0.25, -0.30],
    [-0.41, 1.13, -0.15],
    [-0.25, -0.14, 1.21]
])
b = np.array([0.515, 1.555, 2.780])

# Точний розв'язок
exact_solution = np.linalg.solve(A, b)

# Перевірка вхідних даних
try:
    validate_input(A, b)
except ValueError as e:
    print("Помилка у вхідних даних:", e)
else:
    # Виконання методу Гауса
    try:
        solution_gauss = gauss_method(A, b)
        error_gauss = calculate_error(A, solution_gauss, b)
        coord_accuracy_gauss = calculate_coordinate_accuracy(
            exact_solution, solution_gauss)
        print("Розв'язок системи методом Гауса:", solution_gauss)
        print("Похибка методу Гауса:", error_gauss)
        print("Покоординатна точність методу Гауса:", coord_accuracy_gauss)
    except ValueError as e:
        print("Помилка (метод Гауса):", e)

    # Виконання методу LU-факторизації
    try:
        solution_lu = lu_factorization(A, b)
        error_lu = calculate_error(A, solution_lu, b)
        coord_accuracy_lu = calculate_coordinate_accuracy(
            exact_solution, solution_lu)
        print("Розв'язок системи методом LU-факторизації:", solution_lu)
        print("Похибка методу LU-факторизації:", error_lu)
        print("Покоординатна точність методу LU-факторизації:",
              coord_accuracy_lu)
    except ValueError as e:
        print("Помилка (метод LU-факторизації):", e)


Розв'язок системи методом Гауса: [2.  2.5 3. ]
Похибка методу Гауса: 5.551115123125783e-16
Покоординатна точність методу Гауса: [4.4408921e-16 0.0000000e+00 0.0000000e+00]
Розв'язок системи методом LU-факторизації: [2.  2.5 3. ]
Похибка методу LU-факторизації: 4.577566798522237e-16
Покоординатна точність методу LU-факторизації: [0.0000000e+00 0.0000000e+00 4.4408921e-16]


In [6]:
def test_validate_input():
    # 1. Тест на валідну квадратну матрицю і вектор
    A_valid = np.array([
        [1.02, -0.25, -0.30],
        [-0.41, 1.13, -0.15],
        [-0.25, -0.14, 1.21]
    ])
    b_valid = np.array([0.515, 1.555, 2.780])
    try:
        validate_input(A_valid, b_valid)
        print("Тест 1 пройдено: Валідні дані")
        print("\n"+"-"*50 + "\n")
    except ValueError as e:
        print("Тест 1 не пройдено:", e)

    # 2. Тест на невідповідність розмірів A та b
    A = A_valid
    b_invalid_size = np.array([0.515, 1.555])  # Некоректний розмір
    try:
        validate_input(A, b_invalid_size)
    except ValueError as e:
        assert str(e) == "Кількість рядків у A повинна дорівнювати кількості елементів у b"
        print("Тест 2 пройдено: Некоректний розмір вектора b")
        print("Вхідні дані:")
        print("A =", A)
        print("b =", b_invalid_size)
        print("\n"+"-"*50 + "\n")

    # 3. Тест на неквадратну матрицю A
    A_non_square = np.array([
        [1.02, -0.25],
        [-0.41, 1.13],
        [-0.25, -0.14]
    ])
    b = np.array([0.515, 1.555, 2.780])
    try:
        validate_input(A_non_square, b)
    except ValueError as e:
        assert str(e) == "Матриця A повинна бути квадратною"
        print("Тест 3 пройдено: Неквадратна матриця A")
        print("Вхідні дані:")
        print("A =", A_non_square)
        print("b =", b)
        print("\n"+"-"*50 + "\n")

    # 4. Тест на вироджену матрицю A (визначник = 0)
    A_singular = np.array([
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ])
    try:
        validate_input(A_singular, b)
    except ValueError as e:
        assert str(e) == "Матриця A є виродженою, і система може не мати унікального розв'язку"
        print("Тест 4 пройдено: Вироджена матриця A")
        print("Вхідні дані:")
        print("A =", A_singular)
        print("b =", b)
        print("\n"+"-"*50 + "\n")

    # 5. Тест на невірний тип даних (не масив)
    A_invalid_type = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]  # Не numpy масив
    b_invalid_type = [0.515, 1.555, 2.780]  # Не numpy масив
    try:
        validate_input(A_invalid_type, b_invalid_type)
    except ValueError as e:
        assert str(e) == "Вхідні дані мають бути у вигляді numpy масивів"
        print("Тест 5 пройдено: Некоректний тип даних")
        print("Вхідні дані:")
        print("A =", A_invalid_type)
        print("b =", b_invalid_type)
        print("\n"+"-"*50 + "\n")

    # 6. Тест на нечислові дані в масиві
    A_non_numeric = np.array([
        [1.02, -0.25, -0.30],
        [-0.41, 'invalid', -0.15],  # Некоректне значення
        [-0.25, -0.14, 1.21]
    ])
    try:
        validate_input(A_non_numeric, b)
    except ValueError as e:
        assert str(e) == "Матриця A і вектор b мають містити лише числові значення"
        print("Тест 6 пройдено: Некоректні дані у матриці A")
        print("Вхідні дані:")
        print("A =", A_non_numeric)
        print("b =", b)
        print("\n"+"-"*50 + "\n")

    # 7. Тест на нечисловий вектор b
    b_non_numeric = np.array([0.515, 'invalid', 2.780])  # Некоректне значення
    try:
        validate_input(A_valid, b_non_numeric)
    except ValueError as e:
        assert str(e) == "Матриця A і вектор b мають містити лише числові значення"
        print("Тест 7 пройдено: Некоректні дані у векторі b")
        print("Вхідні дані:")
        print("A =", A_valid)
        print("b =", b_non_numeric)

# Запуск тестів
test_validate_input()


Тест 1 пройдено: Валідні дані

--------------------------------------------------

Тест 2 пройдено: Некоректний розмір вектора b
Вхідні дані:
A = [[ 1.02 -0.25 -0.3 ]
 [-0.41  1.13 -0.15]
 [-0.25 -0.14  1.21]]
b = [0.515 1.555]

--------------------------------------------------

Тест 3 пройдено: Неквадратна матриця A
Вхідні дані:
A = [[ 1.02 -0.25]
 [-0.41  1.13]
 [-0.25 -0.14]]
b = [0.515 1.555 2.78 ]

--------------------------------------------------

Тест 4 пройдено: Вироджена матриця A
Вхідні дані:
A = [[1 2 3]
 [4 5 6]
 [7 8 9]]
b = [0.515 1.555 2.78 ]

--------------------------------------------------

Тест 5 пройдено: Некоректний тип даних
Вхідні дані:
A = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
b = [0.515, 1.555, 2.78]

--------------------------------------------------

Тест 6 пройдено: Некоректні дані у матриці A
Вхідні дані:
A = [['1.02' '-0.25' '-0.3']
 ['-0.41' 'invalid' '-0.15']
 ['-0.25' '-0.14' '1.21']]
b = [0.515 1.555 2.78 ]

---------------------------------------------